# Sequence Models Training

Training sequence models:
- Model 5: Bidirectional LSTM with Attention
- Model 6: Transformer on sequences

In [1]:
import sys
from pathlib import Path
import torch
import torch.nn as nn
import json

PROJECT_ROOT = Path('/Volumes/SSanDisk/SpeechRec-German/without_context_windows')
sys.path.insert(0, str(PROJECT_ROOT))

from models.sequence.bilstm_attention import BiLSTMAttention
from models.sequence.transformer_sequence import TransformerSequence
from utils.training_utils import train_model, evaluate_model
from utils.data_loader import load_data, create_dataloaders

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using MPS device")
else:
    device = torch.device("cpu")

df, spectrograms_dict, feature_cols, feature_scaler, class_weights_dict = load_data(PROJECT_ROOT)
dataloaders = create_dataloaders(df, spectrograms_dict, feature_cols, feature_scaler, class_weights_dict, batch_size=64)

train_loader = dataloaders['sequence']['train']
val_loader = dataloaders['sequence']['val']
test_loader = dataloaders['sequence']['test']

OUTPUT_DIR = PROJECT_ROOT / 'artifacts' / 'b-p_dl_models'
class_weights = torch.tensor([class_weights_dict.get('0', class_weights_dict.get(0, 1.0)), 
                              class_weights_dict.get('1', class_weights_dict.get(1, 1.0))], dtype=torch.float32).to(device)

Using MPS device
Columns in df_phonemes: ['phoneme_id', 'utterance_id', 'phoneme', 'class', 'start_ms', 'end_ms', 'duration_ms', 'audio_path']
Columns in df_features: ['energy_rms', 'energy_rms_std', 'energy_zcr', 'energy_zcr_std', 'spectral_centroid', 'spectral_centroid_std', 'spectral_rolloff', 'spectral_rolloff_std', 'spectral_bandwidth', 'spectral_bandwidth_std', 'formant_f1', 'formant_f2', 'formant_f3', 'formant_f4', 'formant_f1_std', 'formant_f2_std', 'formant_f3_std', 'formant_f4_std', 'spectral_flatness', 'harmonic_noise_ratio', 'zcr_mean', 'energy_cv', 'phoneme_id', 'class', 'duration_ms', 'mfcc_mean_0', 'mfcc_mean_1', 'mfcc_mean_2', 'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5', 'mfcc_mean_6', 'mfcc_mean_7', 'mfcc_mean_8', 'mfcc_mean_9', 'mfcc_mean_10', 'mfcc_mean_11', 'mfcc_mean_12', 'mfcc_std_0', 'mfcc_std_1', 'mfcc_std_2', 'mfcc_std_3', 'mfcc_std_4', 'mfcc_std_5', 'mfcc_std_6', 'mfcc_std_7', 'mfcc_std_8', 'mfcc_std_9', 'mfcc_std_10', 'mfcc_std_11', 'mfcc_std_12', 'delta_mfcc

## Model 5: BiLSTM with Attention

In [2]:
model5 = BiLSTMAttention(input_dim=128, hidden_dim=64, num_layers=2, num_classes=2).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model5.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

save_dir = OUTPUT_DIR / 'sequence_models' / 'bilstm_attention'
save_dir.mkdir(parents=True, exist_ok=True)

history5, best_epoch5 = train_model(model5, train_loader, val_loader, criterion, optimizer, scheduler,
                                    device, num_epochs=50, save_dir=save_dir, model_name='bilstm_attention', early_stopping_patience=10)

checkpoint = torch.load(save_dir / 'best_model.pt')
model5.load_state_dict(checkpoint['model_state_dict'])
test_metrics5, _, _, _ = evaluate_model(model5, test_loader, criterion, device)

with open(save_dir / 'test_metrics.json', 'w') as f:
    json.dump(test_metrics5, f, indent=2)

print(f"Model 5 Test - Acc: {test_metrics5['accuracy']:.4f}, F1: {test_metrics5['f1']:.4f}, ROC-AUC: {test_metrics5['roc_auc']:.4f}")


Epoch 1/50
--------------------------------------------------


Train Loss: 0.3078, Train Acc: 0.8413
Val Loss: 0.9708, Val Acc: 0.3978
Val F1: 0.3209, Val ROC-AUC: 0.9256
✓ New best model saved! (F1: 0.3209)

Epoch 2/50
--------------------------------------------------


Train Loss: 0.2503, Train Acc: 0.8739
Val Loss: 0.3239, Val Acc: 0.8909
Val F1: 0.8897, Val ROC-AUC: 0.9518
✓ New best model saved! (F1: 0.8897)

Epoch 3/50
--------------------------------------------------


Train Loss: 0.2353, Train Acc: 0.8845
Val Loss: 0.5990, Val Acc: 0.6776
Val F1: 0.6868, Val ROC-AUC: 0.9264

Epoch 4/50
--------------------------------------------------


Train Loss: 0.2239, Train Acc: 0.8911
Val Loss: 0.2257, Val Acc: 0.8999
Val F1: 0.9023, Val ROC-AUC: 0.9703
✓ New best model saved! (F1: 0.9023)

Epoch 5/50
--------------------------------------------------


...

Train Loss: 0.1131, Train Acc: 0.9477
Val Loss: 0.1745, Val Acc: 0.9320
Val F1: 0.9331, Val ROC-AUC: 0.9825

Epoch 48/50
--------------------------------------------------


Train Loss: 0.1137, Train Acc: 0.9480
Val Loss: 0.1778, Val Acc: 0.9269
Val F1: 0.9282, Val ROC-AUC: 0.9819

Epoch 49/50
--------------------------------------------------


Train Loss: 0.1115, Train Acc: 0.9482
Val Loss: 0.1749, Val Acc: 0.9310
Val F1: 0.9322, Val ROC-AUC: 0.9827

Epoch 50/50
--------------------------------------------------


Train Loss: 0.1126, Train Acc: 0.9466
Val Loss: 0.1780, Val Acc: 0.9280
Val F1: 0.9294, Val ROC-AUC: 0.9820


Model 5 Test - Acc: 0.9334, F1: 0.9343, ROC-AUC: 0.9839


## Model 6: Transformer Sequence

In [3]:
model6 = TransformerSequence(input_dim=128, d_model=128, nhead=8, num_layers=4, num_classes=2).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.AdamW(model6.parameters(), lr=1e-4, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

save_dir = OUTPUT_DIR / 'sequence_models' / 'transformer_sequence'
save_dir.mkdir(parents=True, exist_ok=True)

history6, best_epoch6 = train_model(model6, train_loader, val_loader, criterion, optimizer, scheduler,
                                    device, num_epochs=50, save_dir=save_dir, model_name='transformer_sequence', early_stopping_patience=10)

checkpoint = torch.load(save_dir / 'best_model.pt')
model6.load_state_dict(checkpoint['model_state_dict'])
test_metrics6, _, _, _ = evaluate_model(model6, test_loader, criterion, device)

with open(save_dir / 'test_metrics.json', 'w') as f:
    json.dump(test_metrics6, f, indent=2)

print(f"Model 6 Test - Acc: {test_metrics6['accuracy']:.4f}, F1: {test_metrics6['f1']:.4f}, ROC-AUC: {test_metrics6['roc_auc']:.4f}")

/Volumes/SSanDisk/SpeechRec-German/venv_pytorch/lib/python3.11/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(



Epoch 1/50
--------------------------------------------------


Train Loss: 0.3376, Train Acc: 0.8268
Val Loss: 0.4900, Val Acc: 0.6992
Val F1: 0.7090, Val ROC-AUC: 0.9468
✓ New best model saved! (F1: 0.7090)

Epoch 2/50
--------------------------------------------------


Train Loss: 0.2299, Train Acc: 0.8852
Val Loss: 0.2211, Val Acc: 0.9125
Val F1: 0.9136, Val ROC-AUC: 0.9697
✓ New best model saved! (F1: 0.9136)

Epoch 3/50
--------------------------------------------------


Train Loss: 0.2031, Train Acc: 0.9000
Val Loss: 0.2263, Val Acc: 0.8935
Val F1: 0.8963, Val ROC-AUC: 0.9705

Epoch 4/50
--------------------------------------------------


...

Train Loss: 0.0813, Train Acc: 0.9621
Val Loss: 0.1520, Val Acc: 0.9455
Val F1: 0.9461, Val ROC-AUC: 0.9867

Epoch 47/50
--------------------------------------------------


Train Loss: 0.0851, Train Acc: 0.9611
Val Loss: 0.1547, Val Acc: 0.9453
Val F1: 0.9459, Val ROC-AUC: 0.9864

Epoch 48/50
--------------------------------------------------


Train Loss: 0.0804, Train Acc: 0.9633
Val Loss: 0.1529, Val Acc: 0.9453
Val F1: 0.9459, Val ROC-AUC: 0.9865

Epoch 49/50
--------------------------------------------------


Train Loss: 0.0797, Train Acc: 0.9628
Val Loss: 0.1545, Val Acc: 0.9460
Val F1: 0.9466, Val ROC-AUC: 0.9864

Early stopping at epoch 49
Best F1: 0.9471 at epoch 39


Model 6 Test - Acc: 0.9477, F1: 0.9479, ROC-AUC: 0.9867
